# Exercise 6

Optionally, you could print a checkerboard and apply the previous steps to a set of images acquired with your own smartphone, webcam or digital camera (thus, calibrating your device).

In [ ]:
# used to reload the imported modules on save
%load_ext autoreload
%autoreload 2

import utils as u

In [ ]:
import matplotlib.pyplot as plt
import plotly.subplots as sp
import plotly.express as px
import numpy as np
import cv2
import os

In [ ]:
# constants
GRID_SIZE = (10,17)
SQUARE_SIZE = 11
IMAGE_SHAPE = (4080, 3072)

# getting the images path
images_pathname = "../our_calibration_images/"
images_path = [os.path.join(images_pathname, imagename) for imagename in os.listdir(images_pathname) if imagename.endswith(".jpg")]

# sorting the lists of strings in numerical order
images_path.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))

## Exercise 1

Calibrate using the Zhang procedure [Zhang, 2002], i.e., find the intrinsic parameters $K$ and, for each image, the pair of $R, t$ (extrinsic)

In [ ]:
V = []
all_H = []

# getting the homographies for each image
for img in images_path:
    H = u.get_homography(img, GRID_SIZE, SQUARE_SIZE)
    all_H.append(H)
    
    v_12 = u.get_v_vector(H, 1, 2)
    v_11 = u.get_v_vector(H, 1, 1)
    v_22 = u.get_v_vector(H, 2, 2)
    
    V.append(v_12)
    V.append(v_11 - v_22)
    
# computing params
V = np.array(V)
K = u.get_intrinsic(V)

all_R = []
all_t = []

# computing extrinsic for each image
for H in all_H:
    R, t = u.get_extrinsic(K, H)
    all_R.append(R)
    all_t.append(t)

print("Example params: \n")
print(f"- K -\n{np.array2string(K, precision=3, suppress_small=True)}\n")
print(f"- R -\n{np.round(all_R[0], 3)}\n")
print(f"- t - \n{np.round(all_t[0], 3)}\n")


## Exercise 2

Choose one of the calibration images and compute the total reprojection error (`Lecture 3, page 45`) for all the grid points (adding a figure with the reprojected points).

In [ ]:

# getting the image and extrinsics
image_index = 1
img_path = images_path[image_index]
R1 = all_R[image_index]
t1 = all_t[image_index]

P = u.get_projection_matrix(K, R1, t1)

corners = u.get_corners(img_path, GRID_SIZE)
projected_corners = []

grid_size_cv2 = tuple(reversed(GRID_SIZE))
for index, corner in enumerate(corners):
    u_coord = corner[0]
    v_coord = corner[1]

    u_index, v_index = np.unravel_index(index, grid_size_cv2)

    # the coordinates of the corner w.r.t. the reference corner at position (0,0) of the corners array
    x_mm = (u_index) * SQUARE_SIZE
    y_mm = (v_index) * SQUARE_SIZE

    point_m = np.array([x_mm, y_mm, 0, 1]) # homogeneous point

    projected_u, projected_v = u.project(point_m, P)[0]
    projected_corners.append((projected_u, projected_v))

# computing Euclidean distance between observed and projected points as error
total_error, mean_error = u.compute_reprojection_error([corners], [projected_corners])
print(f"Error: {total_error:.2f}")
print(f"Mean Error Per Corner: {mean_error:.2f}")

# showing the projected corners
image = cv2.imread(img_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

for corner in projected_corners:
    u_coord, v_coord = int(corner[0]), int(corner[1])
    cv2.circle(image_rgb, (u_coord, v_coord), radius=20, color=(255, 0, 0), thickness=-1)

px.imshow(image_rgb)

We decided also to avarage the error for all the images, in order to make a comparison with the values obtained in the Exercise 6.

In [ ]:
all_corners = []
projected_corners = []

grid_size_cv2 = tuple(reversed(GRID_SIZE))
for image_index, img_path in enumerate(images_path):
    img_path = images_path[image_index]
    R1 = all_R[image_index]
    t1 = all_t[image_index]
    P = u.get_projection_matrix(K, R1, t1)
    corners = u.get_corners(img_path, GRID_SIZE)
    all_corners.append(corners)
    _proj_corners = []
    for index, corner in enumerate(corners):
        u_coord = corner[0]
        v_coord = corner[1]

        u_index, v_index = np.unravel_index(index, grid_size_cv2)

        # the coordinates of the corner w.r.t. the reference corner at position (0,0) of the corners array
        x_mm = (u_index) * SQUARE_SIZE
        y_mm = (v_index) * SQUARE_SIZE

        point_m = np.array([x_mm, y_mm, 0, 1]) # homogeneous point

        projected_u, projected_v = u.project(point_m, P)[0]
        _proj_corners.append((projected_u, projected_v))
    projected_corners.append(_proj_corners)
        
# computing Euclidean distance between observed and projected points as error
total_error, mean_error = u.compute_reprojection_error(all_corners, projected_corners)
print(f"Error: {total_error:.2f}")
print(f"Mean Error Per Corner: {mean_error:.2f}")
normalized_total_error, normalized_mean_error = u.compute_normalized_reprojection_error(all_corners, projected_corners, IMAGE_SHAPE[0], IMAGE_SHAPE[1])
print(f"Normalized Error: {normalized_total_error:.2f}")
print(f"Normalized Mean Error Per Corner: {normalized_mean_error:.7f}")

## Exercise 3

Superimpose an object (for instance, a cylinder as in Fig. 1), to the calibration plane, in 25 images of your choice and check the correctness of the results by visual inspection.

In [ ]:
import random

random.seed(0)
NUM_IMAGES_TO_PROCESS = 25

images_indices = random.sample(range(len(images_path)), NUM_IMAGES_TO_PROCESS)

# 3D parameters of the cylinder (remain fixed for all projections)
radius_mm = 22.0
height_mm = 100.0

# Positioning consistent with the origin of the checkerboard (e.g. 4 squares, 4 squares)
center_x_mm = 5 * SQUARE_SIZE 
center_y_mm = 4 * SQUARE_SIZE
num_sides_cyl = 70 # Cylinder resolution
num_height_slices_cyl = 5

superimposed_image_list = []

for i in images_indices:
    img_path = images_path[i]
    R_i = all_R[i]
    t_i = all_t[i]
    P = u.get_projection_matrix(K, R_i, t_i)
    
    superimposed_image = u.superimpose_cylinder(
        img_path=img_path, 
        P=P,
        radius=radius_mm, 
        height=height_mm, 
        center_x=center_x_mm, 
        center_y=center_y_mm,
        num_sides=num_sides_cyl,
        num_height_slices=num_height_slices_cyl,
        line_thinkness=10
    )
    
    superimposed_image_list.append(superimposed_image)
    
px.imshow(superimposed_image_list[10])

In [ ]:
n = len(superimposed_image_list)
cols = 5
rows = (n + cols - 1) // cols

fig, axes = plt.subplots(rows, cols, figsize=(18, rows * 2))
axes = axes.flatten()

for i, img in enumerate(superimposed_image_list):
    axes[i].imshow(img)
    axes[i].axis('off')

# Hide any unused subplots
for j in range(i + 1, len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.subplots_adjust(wspace=0.0001)
plt.show()

## Exercise 4

Optionally, you could carry out an experiment similar to the one reported in `Lecture 3, p. 65`, plotting the standard deviation of the principal point (entries $u_0$ and $v_0$ of the calibration matrix $K$) as a function of the number of images used for calibration.

In [ ]:
random.seed(0)
max_N_images = 20
N_images = list(range(3, max_N_images + 1))
n_samples = 100

# since V is a stack of two equations per image, use them to compute K
index_to_select = list(range(0, len(V), 2))

u0_std = []
v0_std = []

for n_images in range(3, max_N_images + 1):
    current_sample = 1
    principal_point_coord = []
    while current_sample <= n_samples:
        selected_images = np.array(random.sample(index_to_select, n_images))
        _V = np.concatenate([V[selected_images], V[selected_images + 1]])

        # some matrices could be not positive definite -> no solution
        try:
            _K = u.get_intrinsic(np.array(_V))
        except:
            continue
        
        principal_point_coord.append([_K[0,2], _K[1,2]])
        current_sample += 1
    
    principal_point_coord = np.stack(principal_point_coord)
    _u0_std, _v0_std = principal_point_coord.std(axis=0)
    u0_std.append(_u0_std.item())
    v0_std.append(_v0_std.item())
    
plt.figure(figsize=(10, 6))
plt.plot(N_images, u0_std, marker='o', label='$u_0$ std')
plt.plot(N_images, v0_std, marker='o', label='$v_0$ std')
plt.xlabel('Number of Images')
plt.ylabel('Standard Deviation')
plt.title('Standard Deviation vs Number of Images')
plt.grid(True)
plt.legend()
plt.show()